<a href="https://colab.research.google.com/github/AmonTonui/Final_Year_Project/blob/main/MergeddataRestructuring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Cleaning and Restructuring of the Merged(Education Level and People per room) ONS Dataset

In [ ]:

import pandas as pd

data = pd.read_csv("mergedset.csv")

# Remove the category which includes 0 observations(-8)
data = data[(data['Number of people per room in household (5 categories) Code'] != -8) &
            (data['Highest level of qualification (7 categories) Code'] != -8)]

# Calculate total observations per constituency and group the data by constituencies
total_observations = data.groupby('Westminster Parliamentary constituencies')['Observation'].sum().reset_index()
total_observations.rename(columns={'Observation': 'Total Constituency Observations'}, inplace=True)

# After calculating the total observations, Merge total observations into original dataset to its respective constituency which will be used to divide the number of observations for each constituency(Normalize)
data = data.merge(total_observations, on='Westminster Parliamentary constituencies')

# Normalize each category observations using total observations for each constituency
data['Normalized Observation'] = data['Observation'] / data['Total Constituency Observations']

# Drop unnecessary columns (observation and total constituency observations). We only need the normalized observations per constituency
data.drop(columns=['Observation', 'Total Constituency Observations'], inplace=True)

# Pivot the dataset to have features as separate columns and the constituencies as the rows to analyse the results appropriately
pivoted_data = data.pivot_table(
    index='Westminster Parliamentary constituencies',
    columns=['Number of people per room in household (5 categories)', 'Highest level of qualification (7 categories)'],
    values='Normalized Observation',
    aggfunc='sum'
).reset_index()

# Fill missing values with 0, however there was no missing observations they were already handled by ONS by categorizing them as (-8)
pivoted_data.fillna(0, inplace=True)

# Save the preprocessed dataset
output_file = "Mergednormalized_data.csv"
pivoted_data.to_csv(output_file, index=False)

print(f"Preprocessed data saved to {output_file}")



Preprocessed data saved to final_correctly_normalized_data.csv
